In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/train.csv')
test_df = pd.read_csv('/kaggle/input/test.csv')
submit = pd.read_csv('/kaggle/input/Submission.csv')

In [ ]:
train_df

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
train_df.describe()

In [ ]:
train_df.hist(figsize=(8,8))

In [ ]:
h = plt.hist((np.power(train_df.Item_Outlet_Sales, 1/3)), bins=70)

In [ ]:
# train_df.Item_Outlet_Sales = np.power(train_df.Item_Outlet_Sales, 1/3)

In [ ]:
train_df.info()

In [ ]:
num_cols = []
cat_cols = []

for col in train_df.columns:
    if train_df[col].dtype=='int64' or train_df[col].dtype=='float64':
        num_cols.append(col)
    else:
        cat_cols.append(col)
cat_cols

In [ ]:
for col in cat_cols:
    d = dict()
    d[col] = train_df[col].unique()
    if len(d[col]) < 20:
        print(d)

In [ ]:
print(train_df.Item_Fat_Content.unique())
train_df.replace({'reg':'Regular','LF':'Low Fat','low fat':'Low Fat'},inplace = True)
test_df.replace({'reg':'Regular','LF':'Low Fat','low fat':'Low Fat'},inplace = True)

In [ ]:
train_df['Item_Category'] = train_df['Item_Identifier'].apply(lambda x: x[0:2]).map({'FD':'Food',
                                                              'NC':'Non-Consumable',
                                                              'DR':'Drinks'})
test_df['Item_Category'] = test_df['Item_Identifier'].apply(lambda x: x[0:2]).map({'FD':'Food',
                                                              'NC':'Non-Consumable',
                                                              'DR':'Drinks'})
cat_cols.append('Item_Category')

In [ ]:
for col in cat_cols[1:]:
    sns.barplot(data=train_df, x=col, y='Item_Outlet_Sales')
    t = plt.xticks(rotation=90)
    plt.show()

In [ ]:
sns.barplot(data=train_df, x='Outlet_Establishment_Year', y='Item_Outlet_Sales')

In [ ]:
#x = 70, 138, 202
sns.scatterplot(data=train_df, x='Item_MRP', y='Item_Outlet_Sales')
plt.plot([70, 70], [0,12000])
plt.plot([137, 137], [0,12000])
plt.plot([203, 203], [0,12000])

* 0-70 
* 71-138
* 139-202
* 203-..

In [ ]:
# def func(x):
#     if x<71:
#         return 1
#     elif x<139:
#         return 2
#     elif x<203:
#         return 3
#     else:
#         return 4
# train_df['Item_MRP_Classes'] = train_df.Item_MRP.apply(lambda x:func(x))
# test_df['Item_MRP_Classes'] = test_df.Item_MRP.apply(lambda x:func(x))

In [ ]:
train_df.columns

In [ ]:
corr = train_df.corr()
corr['Item_Outlet_Sales']

In [ ]:
_ = train_df.groupby(['Item_Category', 'Item_Fat_Content', 'Item_Type']).describe()['Item_Weight']['mean'].reindex()
_

In [ ]:
for i in range(len(_.index)):
    row = _.index[i]
    train_df.loc[(train_df['Item_Weight'].isnull()) & (train_df['Item_Category']==row[0]) & (train_df['Item_Fat_Content']==row[1]) & (train_df['Item_Type']==row[2]), 'Item_Weight'] = _.values[i]
    
train_df['Item_Weight'] = train_df['Item_Weight'].fillna((train_df['Item_Weight']).mean())

for i in range(len(_.index)):
    row = _.index[i]
    test_df.loc[(test_df['Item_Weight'].isnull()) & (test_df['Item_Category']==row[0]) & (test_df['Item_Fat_Content']==row[1]) & (test_df['Item_Type']==row[2]), 'Item_Weight'] = _.values[i]

test_df['Item_Weight'] = test_df['Item_Weight'].fillna((test_df['Item_Weight']).mean())

In [ ]:
train_df.groupby(['Outlet_Location_Type','Outlet_Identifier'])['Outlet_Type'].value_counts()

In [ ]:
train_df.groupby(['Outlet_Location_Type','Outlet_Identifier', 'Outlet_Type'])['Outlet_Size'].value_counts()

So Outlets OUT017, OUT045, OUT010 have outlet_size as missing values.
Now
1. Since OUT010 is a Grocery Store of tier 3 we can fill with medium in outlet_size
2. Since OUT017 and OUT045 are of supermarkettype 1 of tier 2, it's outlet_size should be small

In [ ]:
train_df[train_df['Outlet_Identifier']=='OUT010'] = \
    train_df[train_df['Outlet_Identifier']=='OUT010'].fillna('Medium', axis=0)
train_df[train_df['Outlet_Identifier']=='OUT045'] = \
    train_df[train_df['Outlet_Identifier']=='OUT045'].fillna('Small', axis=0)
train_df[train_df['Outlet_Identifier']=='OUT017'] = \
    train_df[train_df['Outlet_Identifier']=='OUT017'].fillna('Small', axis=0)

test_df[test_df['Outlet_Identifier']=='OUT010'] = \
    test_df[test_df['Outlet_Identifier']=='OUT010'].fillna('Medium', axis=0)
test_df[test_df['Outlet_Identifier']=='OUT045'] = \
    test_df[test_df['Outlet_Identifier']=='OUT045'].fillna('Small', axis=0)
test_df[test_df['Outlet_Identifier']=='OUT017'] = \
    test_df[test_df['Outlet_Identifier']=='OUT017'].fillna('Small', axis=0)

In [ ]:
cat_cols

In [ ]:
dum_cols = ['Item_Fat_Content', 'Outlet_Size',
 'Outlet_Location_Type',
 'Outlet_Type',
 'Item_Category']

categ_cols = ['Item_Type','Outlet_Identifier']

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

# train_df = pd.get_dummies(data=train_df, columns=dum_cols, drop_first=True)
# test_df = pd.get_dummies(data=test_df, columns=dum_cols, drop_first=True)

In [ ]:
train_df.head(2)

In [ ]:
train_df['fat_per_weight'] = train_df['Item_Fat_Content']/train_df['Item_Weight']
test_df['fat_per_weight'] = test_df['Item_Fat_Content']/test_df['Item_Weight']

In [ ]:
train_df.columns

In [ ]:
# features = ['Item_Weight', 'Item_Visibility', 'Item_Type',
#        'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year',
#        'Item_Fat_Content_Regular', 'Outlet_Size_Medium',
#        'Outlet_Size_Small', 'Outlet_Location_Type_Tier 2',
#        'Outlet_Location_Type_Tier 3', 'Outlet_Type_Supermarket Type1',
#        'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3',
#        'Item_Category_Food', 'Item_Category_Non-Consumable', 'Item_Outlet_Sales']
# corr = train_df[features].corr()
# features = features[:-1]
# corr['Item_Outlet_Sales']

In [ ]:
features = ['Item_Fat_Content', 'Item_Visibility', 'fat_per_weight', 
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
        'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Establishment_Year',
       'Outlet_Type', 'Item_Category', 'Item_Outlet_Sales']
corr = train_df[features].corr()
features = features[:-1]
corr['Item_Outlet_Sales']

In [ ]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train_df[features], train_df['Item_Outlet_Sales'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import mean_squared_error as mse
def get_e(model, df, target):
    pred = model.predict(df)
    print(len(pred[pred<=0]))
    pred[pred<=0] = 2200
    e = mse(target, pred)
    print(np.sqrt(e))

def preds_to_csv(model, name):
    pred = model.predict(test_df[features])
    pred[pred<=0] = 2200
    submit['Item_Outlet_Sales'] = pred
    submit.to_csv('big_mart_sales_'+name+'.csv', index=False)
#     pd.DataFrame({'Item_Identifier':test_df.Item_Identifier, 'Outlet_Identifier':test_df.Outlet_Identifier, 'Item_Outlet_Sales':pred}).to_csv(name+'_big_mart_sales.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
import lightgbm as lgb

In [ ]:
rm -r *.csv

In [ ]:
# rfr = RandomForestRegressor()
# rfr.fit(train_df[features], train_df.Item_Outlet_Sales)
# get_e(rfr, val_X, val_y)
# preds_to_csv(rfr, 'rfr')

In [ ]:
# from sklearn.model_selection import KFold

# gbr = GradientBoostingRegressor()
# X = train_df[features].values
# y = train_df['Item_Outlet_Sales'].values
# kf = KFold(n_splits=3, random_state=42)
# for train_index, test_index in kf.split(X):
#     X_train, X_val = X[train_index], X[test_index]
#     y_train, y_val = y[train_index], y[test_index]
#     gbr.fit(X_train, y_train)
#     get_e(gbr, X_val, y_val)

# get_e(gbr, val_X, val_y)
# preds_to_csv(gbr, 'k_fold_gbr')

In [ ]:
gbr = GradientBoostingRegressor()
gbr.fit(train_df[features], train_df.Item_Outlet_Sales)
get_e(gbr, val_X, val_y)
preds_to_csv(gbr, 'gbr')

In [ ]:
gbr.get_params()

In [ ]:
parameters = {
    "learning_rate": [0.01, 0.05, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 1, 6),
    "min_samples_leaf": np.linspace(0.1, 0.5, 5),
    "max_depth":[2,3,4,5,8],
    "max_features":["auto", "log2","sqrt"],
    "criterion": ["friedman_mse"],
    "n_estimators":[50, 75, 100, 125, 150]
    }

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(gbr, parameters, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(train_df[features], train_df.Item_Outlet_Sales)
get_e(grid_search, val_X, val_y)
preds_to_csv(grid_search, 'grid_search')

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.best_params_

In [ ]:
# lgbr = lgb.LGBMRegressor()
# lgbr.fit(train_df[features], train_df.Item_Outlet_Sales)
# get_e(lgbr, val_X, val_y)
# preds_to_csv(lgbr, 'lgbr')

In [ ]:
# lr = LinearRegression()
# lr.fit(train_X, train_y)
# get_e(lr, val_X, val_y)
# preds_to_csv(lr, 'lr')

In [ ]:
# xgbr = xgb.XGBRegressor()
# xgbr.fit(train_X, train_y)
# get_e(xgbr, val_X, val_y)
# preds_to_csv(xgbr, 'xgbr')

In [ ]:
# etr = ExtraTreesRegressor()
# etr.fit(train_X, train_y)
# get_e(etr, val_X, val_y)
# preds_to_csv(etr, 'etr')

GBR with whole train data gives a good score

In [ ]:
from tpot import TPOTRegressor

tpr = TPOTRegressor(generations=100, population_size=10, n_jobs=-1, early_stop=20, verbosity=3, random_state=42)
tpr.fit(train_df[features], train_df.Item_Outlet_Sales)
get_e(tpr, val_X, val_y)
preds_to_csv(tpr, 'tpr')

In [ ]:
tpr.fitted_pipeline_